In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [10]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x).cuda()
    uxx = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    xe1 = torch.rand(Nb, dimension).cuda()  
    xe1[:, 4] = torch.zeros(Nb)
    xe2 = torch.rand(Nb, dimension).cuda()  
    xe2[:, 4] = torch.ones(Nb)
    xf1 = torch.rand(Nb, dimension).cuda() 
    xf1[:, 5] = torch.zeros(Nb)
    xf2 = torch.rand(Nb, dimension).cuda() 
    xf2[:, 5] = torch.ones(Nb)
    xg1 = torch.rand(Nb, dimension).cuda() 
    xg1[:, 6] = torch.zeros(Nb)
    xg2 = torch.rand(Nb, dimension).cuda() 
    xg2[:, 6] = torch.ones(Nb)
    xh1 = torch.rand(Nb, dimension).cuda() 
    xh1[:, 7] = torch.zeros(Nb)
    xh2 = torch.rand(Nb, dimension).cuda() 
    xh2[:, 7] = torch.ones(Nb)
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) + model(xa1) - g(xa1))**2).cuda()
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) + model(xa2) - g(xa2))**2).cuda()
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) + model(xb1) - g(xb1))**2).cuda()
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) + model(xb2) - g(xb2))**2).cuda()
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) + model(xc1) - g(xc1))**2).cuda()
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) + model(xc2) - g(xc2))**2).cuda()
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) + model(xd1) - g(xd1))**2).cuda()
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) + model(xd2) - g(xd2))**2).cuda()
    uxe1 = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 * (1.0) + model(xe1) - g(xe1))**2).cuda()
    uxe2 = (((model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2 * (-1.0) + model(xe2) - g(xe2))**2).cuda()
    uxf1 = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 * (1.0) + model(xf1) - g(xf1))**2).cuda()
    uxf2 = (((model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2 * (-1.0) + model(xf2) - g(xf2))**2).cuda()
    uxg1 = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 * (1.0) + model(xg1) - g(xg1))**2).cuda()
    uxg2 = (((model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2 * (-1.0) + model(xg2) - g(xg2))**2).cuda()
    uxh1 = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 * (1.0) + model(xh1) - g(xh1))**2).cuda()
    uxh2 = (((model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2 * (-1.0) + model(xh2) - g(xh2))**2).cuda()
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2 + uxe1 + uxe2 + uxf1 + uxf2 + uxg1 + uxg2 + uxh1 + uxh2)[:,0])/xa1.size()[0]
    lambda1 = 10.0
    return part_1 + lambda1 * part_2 / 16

In [11]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [12]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [13]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [14]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [15]:
import torch.optim as optim
import torch.nn as nn
import time

In [16]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [17]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [18]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_8D_Robin.npy", error_save)
np.save("DGM_relative_error_8D_Robin.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(7564.8464, device='cuda:0')
current relative error is:  tensor(2.2659, device='cuda:0')
current epoch is:  50
current loss is:  tensor(1706.2420, device='cuda:0')
current relative error is:  tensor(1.0425, device='cuda:0')
current epoch is:  100
current loss is:  tensor(1306.8066, device='cuda:0')
current relative error is:  tensor(0.9102, device='cuda:0')
current epoch is:  150
current loss is:  tensor(874.6214, device='cuda:0')
current relative error is:  tensor(0.8086, device='cuda:0')
current epoch is:  200
current loss is:  tensor(416.4014, device='cuda:0')
current relative error is:  tensor(0.9202, device='cuda:0')
current epoch is:  250
current loss is:  tensor(193.7749, device='cuda:0')
current relative error is:  tensor(0.8573, device='cuda:0')
current epoch is:  300
current loss is:  tensor(151.9144, device='cuda:0')
current relative error is:  tensor(0.7846, device='cuda:0')
current epoch is:  350
current loss is:  tensor(123.167

current epoch is:  3050
current loss is:  tensor(21.9019, device='cuda:0')
current relative error is:  tensor(0.3438, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(20.4355, device='cuda:0')
current relative error is:  tensor(0.3283, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(22.2109, device='cuda:0')
current relative error is:  tensor(0.3242, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(20.5000, device='cuda:0')
current relative error is:  tensor(0.3104, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(20.0411, device='cuda:0')
current relative error is:  tensor(0.3020, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(19.8892, device='cuda:0')
current relative error is:  tensor(0.3091, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(19.2539, device='cuda:0')
current relative error is:  tensor(0.3020, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(18.729

current epoch is:  6100
current loss is:  tensor(4.1845, device='cuda:0')
current relative error is:  tensor(0.1965, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(3.4905, device='cuda:0')
current relative error is:  tensor(0.1769, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(4.5740, device='cuda:0')
current relative error is:  tensor(0.1742, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(3.5048, device='cuda:0')
current relative error is:  tensor(0.1770, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(3.5358, device='cuda:0')
current relative error is:  tensor(0.1725, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(3.3684, device='cuda:0')
current relative error is:  tensor(0.1692, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(3.6066, device='cuda:0')
current relative error is:  tensor(0.1600, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(3.8417, devic

current epoch is:  9200
current loss is:  tensor(1.8897, device='cuda:0')
current relative error is:  tensor(0.1152, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(1.8818, device='cuda:0')
current relative error is:  tensor(0.1189, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(1.9006, device='cuda:0')
current relative error is:  tensor(0.1120, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(1.9261, device='cuda:0')
current relative error is:  tensor(0.1132, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(2.0905, device='cuda:0')
current relative error is:  tensor(0.1204, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(1.8285, device='cuda:0')
current relative error is:  tensor(0.1120, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(1.8577, device='cuda:0')
current relative error is:  tensor(0.1137, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(1.7203, devic

current epoch is:  12250
current loss is:  tensor(2.0146, device='cuda:0')
current relative error is:  tensor(0.1012, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(1.4093, device='cuda:0')
current relative error is:  tensor(0.0971, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(1.2771, device='cuda:0')
current relative error is:  tensor(0.0955, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(1.6269, device='cuda:0')
current relative error is:  tensor(0.0978, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(1.3058, device='cuda:0')
current relative error is:  tensor(0.0983, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(2.6821, device='cuda:0')
current relative error is:  tensor(0.1022, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(1.4459, device='cuda:0')
current relative error is:  tensor(0.0983, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(1.056

...

current epoch is:  36650
current loss is:  tensor(0.4883, device='cuda:0')
current relative error is:  tensor(0.0632, device='cuda:0')
current epoch is:  36700
current loss is:  tensor(0.3492, device='cuda:0')
current relative error is:  tensor(0.0656, device='cuda:0')
current epoch is:  36750
current loss is:  tensor(0.5341, device='cuda:0')
current relative error is:  tensor(0.0624, device='cuda:0')
current epoch is:  36800
current loss is:  tensor(1.9829, device='cuda:0')
current relative error is:  tensor(0.0742, device='cuda:0')
current epoch is:  36850
current loss is:  tensor(0.4929, device='cuda:0')
current relative error is:  tensor(0.0650, device='cuda:0')
current epoch is:  36900
current loss is:  tensor(0.3628, device='cuda:0')
current relative error is:  tensor(0.0643, device='cuda:0')
current epoch is:  36950
current loss is:  tensor(1.1366, device='cuda:0')
current relative error is:  tensor(0.0694, device='cuda:0')
current epoch is:  37000
current loss is:  tensor(0.370

current epoch is:  39700
current loss is:  tensor(0.3300, device='cuda:0')
current relative error is:  tensor(0.0636, device='cuda:0')
current epoch is:  39750
current loss is:  tensor(1.1675, device='cuda:0')
current relative error is:  tensor(0.0703, device='cuda:0')
current epoch is:  39800
current loss is:  tensor(0.3258, device='cuda:0')
current relative error is:  tensor(0.0674, device='cuda:0')
current epoch is:  39850
current loss is:  tensor(0.8499, device='cuda:0')
current relative error is:  tensor(0.0708, device='cuda:0')
current epoch is:  39900
current loss is:  tensor(0.3224, device='cuda:0')
current relative error is:  tensor(0.0642, device='cuda:0')
current epoch is:  39950
current loss is:  tensor(0.3230, device='cuda:0')
current relative error is:  tensor(0.0638, device='cuda:0')
current epoch is:  40000
current loss is:  tensor(0.3648, device='cuda:0')
current relative error is:  tensor(0.0612, device='cuda:0')
current epoch is:  40050
current loss is:  tensor(0.379

current epoch is:  42750
current loss is:  tensor(0.2888, device='cuda:0')
current relative error is:  tensor(0.0604, device='cuda:0')
current epoch is:  42800
current loss is:  tensor(0.3235, device='cuda:0')
current relative error is:  tensor(0.0629, device='cuda:0')
current epoch is:  42850
current loss is:  tensor(0.7334, device='cuda:0')
current relative error is:  tensor(0.0674, device='cuda:0')
current epoch is:  42900
current loss is:  tensor(0.9212, device='cuda:0')
current relative error is:  tensor(0.0657, device='cuda:0')
current epoch is:  42950
current loss is:  tensor(0.3889, device='cuda:0')
current relative error is:  tensor(0.0628, device='cuda:0')
current epoch is:  43000
current loss is:  tensor(0.3698, device='cuda:0')
current relative error is:  tensor(0.0616, device='cuda:0')
current epoch is:  43050
current loss is:  tensor(0.2943, device='cuda:0')
current relative error is:  tensor(0.0617, device='cuda:0')
current epoch is:  43100
current loss is:  tensor(0.975

current epoch is:  45800
current loss is:  tensor(0.2757, device='cuda:0')
current relative error is:  tensor(0.0583, device='cuda:0')
current epoch is:  45850
current loss is:  tensor(0.3500, device='cuda:0')
current relative error is:  tensor(0.0575, device='cuda:0')
current epoch is:  45900
current loss is:  tensor(0.3281, device='cuda:0')
current relative error is:  tensor(0.0597, device='cuda:0')
current epoch is:  45950
current loss is:  tensor(0.3415, device='cuda:0')
current relative error is:  tensor(0.0599, device='cuda:0')
current epoch is:  46000
current loss is:  tensor(0.9239, device='cuda:0')
current relative error is:  tensor(0.0775, device='cuda:0')
current epoch is:  46050
current loss is:  tensor(0.3205, device='cuda:0')
current relative error is:  tensor(0.0618, device='cuda:0')
current epoch is:  46100
current loss is:  tensor(0.2913, device='cuda:0')
current relative error is:  tensor(0.0591, device='cuda:0')
current epoch is:  46150
current loss is:  tensor(0.292

current epoch is:  48850
current loss is:  tensor(0.2558, device='cuda:0')
current relative error is:  tensor(0.0560, device='cuda:0')
current epoch is:  48900
current loss is:  tensor(0.3754, device='cuda:0')
current relative error is:  tensor(0.0639, device='cuda:0')
current epoch is:  48950
current loss is:  tensor(0.2559, device='cuda:0')
current relative error is:  tensor(0.0583, device='cuda:0')
current epoch is:  49000
current loss is:  tensor(0.3388, device='cuda:0')
current relative error is:  tensor(0.0633, device='cuda:0')
current epoch is:  49050
current loss is:  tensor(0.2841, device='cuda:0')
current relative error is:  tensor(0.0573, device='cuda:0')
current epoch is:  49100
current loss is:  tensor(1.2357, device='cuda:0')
current relative error is:  tensor(0.0708, device='cuda:0')
current epoch is:  49150
current loss is:  tensor(0.2722, device='cuda:0')
current relative error is:  tensor(0.0577, device='cuda:0')
current epoch is:  49200
current loss is:  tensor(0.284

In [19]:
x_test = torch.rand(10, 8)
torch.abs(model(x) - u_ex(x))

tensor([[0.1712],
        [0.0724],
        [0.0394],
        ...,
        [0.2796],
        [0.1535],
        [0.0019]], device='cuda:0', grad_fn=<AbsBackward>)